In [3]:
from data_utils import *
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

C:\Users\user\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
dtype = torch.FloatTensor

In [5]:
with open('word_dict.pickle', 'rb') as handle:
    word_dict = pickle.load(handle)

In [6]:
train_x, train_y = build_word_dataset('train', word_dict,50) ##최대 50 token 나머지는 패딩

C:\Users\user\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [7]:
test_x, test_y = build_word_dataset('test', word_dict,50)

C:\Users\user\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [2]:
train_x

NameError: name 'train_x' is not defined

In [176]:
len(word_dict)

49898

In [182]:
embedding_size = 256 
sequence_length = 50
num_classes = 2
filter_sizes = [2,3,4] # n-gram 다양한 filtersize로 filter를 적용하겠다. 
num_filters = 10
vocab_size = len(word_dict)
num_epochs = 5
batch_size = 64

In [183]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN,self).__init__()
        
        self.num_filters_total = num_filters * len(filter_sizes)
        self.W = nn.Parameter(torch.empty(vocab_size,embedding_size).uniform_(-1,1)).type(dtype)
        self.Weight = nn.Parameter(torch.empty(self.num_filters_total,num_classes).uniform_(-1,1)).type(dtype)
        self.Bias = nn.Parameter(0.1 * torch.ones([num_classes])).type(dtype)

        
    def forward(self,X):
        embedded_chars = self.W[X] # [batch_size, sequence_length, embedding_size]
        embedded_chars = embedded_chars.unsqueeze(1) # add channel(=1) [batch, channel(=1), sequence_length, embedding_size]
        
        pooled_outputs =[]
        for filter_size in filter_sizes:
            # conv : [input_channel(=1), output_channel(=3), (filter_height, filter_width), bias_option]
            conv = nn.Conv2d(1,num_filters,(filter_size, embedding_size), bias = True)(embedded_chars) ## conv2d를 embedded_chars에 적용
            h = F.relu(conv)
            # mp : ((filter_height, filter_width))
            mp = nn.MaxPool2d((sequence_length-filter_size+1,1))
            # pooled : [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3)]
            pooled = mp(h).permute(0,3,2,1)
            pooled_outputs.append(pooled)
            
        h_pool = torch.cat(pooled_outputs,len(filter_sizes)) ## channel을 늘인거 [batch_size(=6), output_height(=1), output_width(=1),output_channel(=3)*3]
        h_pool_flat = torch.reshape(h_pool, [-1,self.num_filters_total]) # [batch_size(=6), output_height * output_width * (output_channel *3)] 여기서 output_channel *3이 결국 num_filters_total이 된다
        model = torch.mm(h_pool_flat,self.Weight) + self.Bias # [batch_size, num_classes]    
        return model

In [184]:
model  = TextCNN()

In [187]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [188]:
# Training


train_batches = batch_iter(train_x,train_y,batch_size,num_epochs)
num_batches_per_epochs = (len(train_x)-1) // batch_size +1
step = 0
running_loss = 0
    
for x_batch,y_batch in train_batches:
        
    #FeedForward
        
    step += 1
    x_batch = Variable(torch.LongTensor(x_batch))
    y_batch = Variable(torch.LongTensor(y_batch))
    output = model(x_batch)
    loss = criterion(output,y_batch)
        
    #Backprop and update weight
       
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    #loss 출력
    running_loss += loss.item()
    if (step % 100 == 0):
        print("Loss: {}".format(running_loss/100),"Epoch: {}".format(epoch+1),'Step: {}'.format(step))
        running_loss = 0
            

        
    

Loss: 0.5632700380682946 Epoch: 10 Step: 100
Loss: 0.5584706944227219 Epoch: 10 Step: 200
Loss: 0.5652339935302735 Epoch: 10 Step: 300
Loss: 0.5741479927301407 Epoch: 10 Step: 400
Loss: 0.5643043798208237 Epoch: 10 Step: 500
Loss: 0.5453199142217636 Epoch: 10 Step: 600
Loss: 0.5639192405343055 Epoch: 10 Step: 700
Loss: 0.5639493584632873 Epoch: 10 Step: 800
Loss: 0.5541241294145585 Epoch: 10 Step: 900
Loss: 0.5719846737384796 Epoch: 10 Step: 1000
Loss: 0.5451348197460174 Epoch: 10 Step: 1100
Loss: 0.5486654365062713 Epoch: 10 Step: 1200
Loss: 0.55373967140913 Epoch: 10 Step: 1300
Loss: 0.5629369115829468 Epoch: 10 Step: 1400
Loss: 0.5485244327783585 Epoch: 10 Step: 1500
Loss: 0.5639682605862617 Epoch: 10 Step: 1600
Loss: 0.5548910310864449 Epoch: 10 Step: 1700
Loss: 0.5642914342880249 Epoch: 10 Step: 1800
Loss: 0.5571119791269302 Epoch: 10 Step: 1900
Loss: 0.55411581158638 Epoch: 10 Step: 2000
Loss: 0.5721792334318161 Epoch: 10 Step: 2100
Loss: 0.5631011876463891 Epoch: 10 Step: 2200
L